# 05 - Dictionaries (Teams & Players)
Génération et aperçu des dictionnaires équipes/joueurs.


In [1]:
import sys
from pathlib import Path

def _find_root():
    cand = Path.cwd()
    for c in [cand, *cand.parents]:
        if (c / 'src').exists() and (c / 'data').exists():
            return c
    return cand

ROOT = _find_root()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

processed_dir = ROOT / 'data/processed'
parquet_paths = [processed_dir / f for f in ['matches.parquet', 'players.parquet', 'objectives.parquet', 'teamfights.parquet']]
missing = [p for p in parquet_paths if not p.exists()]
if missing:
    raise FileNotFoundError(f"Parquets manquants: {missing}. Regenerer avec `make parquet`.")

import polars as pl
from src.dota_data import read_processed_tables, build_team_dictionary, build_player_dictionary

tables = read_processed_tables(processed_dir)
teams = build_team_dictionary(tables['matches'])
players = build_player_dictionary(tables['players'])

out_dir = ROOT / 'data/dictionaries'
out_dir.mkdir(parents=True, exist_ok=True)
teams_path = out_dir / 'teams.parquet'
players_path = out_dir / 'players.parquet'
teams.write_parquet(teams_path)
players.write_parquet(players_path)

teams.shape, players.shape


((690, 5), (2012, 4))

In [2]:
teams.head(10)


team_id,name,tag,logo_url,side_sampled
i64,str,str,str,str
9247798,"""Team Hryvnia""","""PassionUA""","""https://cdn.steamusercontent.c…","""dire"""
9683757,"""Chicken N Friends""","""CNF""","""https://cdn.steamusercontent.c…","""dire"""
9308478,"""Los Magicards""","""Magic""","""2725923848512""","""dire"""
9724550,"""Red Submarine""","""RedSub""","""https://cdn.steamusercontent.c…","""dire"""
9297546,"""MAG.Nirvana""","""""","""https://steamusercontent-a.aka…","""radiant"""
9652269,"""PERRO NEGRO""","""PRN""","""https://cdn.steamusercontent.c…","""dire"""
9187066,"""Team Tickles""","""TT""","""https://steamusercontent-a.aka…","""dire"""
9885654,"""Team Aureus""","""""","""https://cdn.steamusercontent.c…","""dire"""
9824144,"""Quantum""","""""","""4294967295""","""dire"""


In [3]:
players.head(10)


account_id,matches_played,personaname,name
i64,u32,str,str
86698277,626,"""not human (животное)""","""33"""
183719386,625,"""SumaoKy-""","""AMMAR_THE_F"""
10366616,618,"""Sneyking""","""Sneyking"""
25907144,618,"""ik lavet penge""","""Cr1t-"""
100058342,615,"""Falcons Malr1ne prime""","""skiter"""
136829091,606,"""NongMon""","""Whitemon"""
77490514,602,"""Boxi""","""Boxi"""
185059559,601,"""-""","""Till The End"""
116249155,599,"""nefrit""","""nefrit"""
